# Preparing Inputs for *segger* from Different iST Platforms

Because data formats differ across platforms, here we provide guides to convert raw spatial transcriptomics data into a format compatible with ***segger***. 

**NOTE**: Currently, we provide instructions for preprocessing standard outputs for the platforms listed below:
- 10x Genomics Xenium
- NanoString CosMX
- Vizgen MERSCOPE

However, as long as your data adheres to the input format described below, you can run Segger on custom or unsupported platforms as well.

---

### Required Formatting for *segger* Inputs

At minimum, ***segger*** expects the following two inputs:

1. **Transcripts** (GeoParquet with point geometry). Each row must include:
    - The gene or feature label for the transcript.
    - A segmentation ID matching the index of one element in the boundary polygon file.
    - An X and Y location.
2. **Boundaries** (GeoParquet with polygon geometry). Each row must include:
    - A GeoArrow polygon geometry column (coordinates must be in the same space as the transcript coordinates).

In [54]:
# Main
from segger.io.cosmx import get_cosmx_polygons
from segger.io._enum import Filter_Substrings
from segger.io import _utils as utils
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

# Plotting
from matplotlib.collections import PolyCollection
from matplotlib import pyplot as plt
plt.style.use('moorman')

# Warnings
import warnings
from pandas.errors import DtypeWarning
warnings.filterwarnings("ignore", category=DtypeWarning)

## Nanostring CosMX

Segger uses nuclear boundaries as input, but CosMX datasets often only
provide labeled cell and compartment masks. This section first extracts nuclear
boundaries from CosMX output.

The script expects the following structure in the CosMX output directory:

```
data_dir/
├── *_fov_positions_file.csv
├── CellLabels/
│   ├── CellLabels_F001.tif
│   ├── CellLabels_F002.tif
│   └── ...
└── CompartmentLabels/
    ├── CompartmentLabels_F001.tif
    ├── CompartmentLabels_F002.tif
    └── ...
```

These files are produced by NanoString’s CosMX pipeline. Nuclear masks are
extracted as the regions where the compartment label equals `1` (nucleus),
intersected with cell labels.

In [12]:
# Sample base directory
data_dir = Path('/data1/peerd/moormana/data/segger/segmentation_benchmarking'
                '/cosmx_human_pancreas_healthy/data/raw')

# Input file directories and paths
cell_lbl_dir = data_dir / 'CellLabels'
comp_lbl_dir = data_dir / 'CompartmentLabels'
fov_path = data_dir / 'Pancreas_fov_positions_file.csv'
tx_path = data_dir / 'Pancreas_tx_file.csv'

# Boundary type ('cell' or 'nucleus')
bd_type = 'cell'

# Output file paths
tx_path_out = data_dir / 'transcripts_geo.parquet'
bd_path_out = data_dir / f'{bd_type}_boundaries_geo.parquet'

In [13]:
# Read in CosMX nucleus boundaries and convert to GeoParquet
bd = get_cosmx_polygons(cell_lbl_dir, comp_lbl_dir, fov_path, bd_type)
bd.to_parquet(bd_path_out, write_covering_bbox=True,
              geometry_encoding='geoarrow')

# 18.3s for 47499 boundaries

In [15]:
# If transcripts already exists, just append boundary assignments
if tx_path_out.exists():
    tx = gpd.read_parquet(tx_path_out)

# Else, read in CosMX transcript data and convert to GeoParquet
else:
    tx = pd.read_csv(tx_path) 
    pts = gpd.points_from_xy(tx.x_global_px, tx.y_global_px)
    tx = gpd.GeoDataFrame(tx, geometry=pts)

# Assign transcripts to boundaries using spatial join
tx = gpd.sjoin(tx, bd, how='left', predicate='intersects')
tx.rename({'index_right': f'{bd_type}_boundaries_id'}, inplace=True, axis=1)

# Write to file
tx.to_parquet(tx_path_out, write_covering_bbox=False,
              geometry_encoding='geoarrow')

# Performance: 3m 8.8s for 65794659 transcripts

In [16]:
# Write counts matrix from transcript assignments (nuclei only)
if bd_type == 'nucleus':

    ad = utils.transcripts_to_anndata(
        tx, cell_label=f'{bd_type}_boundaries_id', gene_label='target',
        coordinate_labels=['x_global_px', 'y_global_px'])

    ad.write_h5ad(data_dir / f'{bd_type}_boundaries.h5ad')

# Performance: 14.5s for 47487 x 21731 AnnData

## 10X Xenium

This section expects the files listed below in a Xenium output directory:
```
data_dir/
├── transcripts.parquet
└── nucleus_boundaries.parquet
```
By default, these files are produced by 10x Genomics’ Xenium pipeline in the correct directory structure.

In [76]:
# Sample base directory
data_dir = Path('/data1/peerd/moormana/data/segger/preprint'
                '/xenium_human_colon_healthy')

# Boundary type ('cell' or 'nucleus')
bd_type = 'nucleus'

# Input file paths
tx_path = data_dir / 'transcripts.parquet'
bd_path = data_dir / f'{bd_type}_boundaries.parquet'

# Output file paths
tx_path_out = data_dir / 'transcripts_geo.parquet'
bd_path_out = data_dir / f'{bd_type}_boundaries_geo.parquet'

In [66]:
# Read in 10X Xenium boundaries and convert to GeoParquet
bd = pd.read_parquet(bd_path)
bd = utils.contours_to_polygons(bd['vertex_x'], bd['vertex_y'], bd['cell_id'])
bd.to_parquet(bd_path_out, write_covering_bbox=True,
              geometry_encoding='geoarrow')

# Performance: 3.6s for 275822 boundaries

In [67]:
# If transcript file exists, just append boundary assignments
if tx_path_out.exists():
    tx = gpd.read_parquet(tx_path_out)

else:
    # Read in 10X Xenium transcript data and filter
    tx = pd.read_parquet(tx_path)
    mask = tx.qv.ge(20)
    pat = '|'.join(Filter_Substrings.tenx_xenium.value)
    mask &= ~tx.feature_name.str.contains(pat)
    tx = tx[mask]

    # Convert to GeoParquet
    pts = gpd.points_from_xy(tx.x_location, tx.y_location)
    tx = gpd.GeoDataFrame(tx, geometry=pts)

# Assign transcripts to boundaries using spatial join
tx = gpd.sjoin(tx, bd, how='left', predicate='intersects')
tx.rename({'index_right': f'{bd_type}_boundaries_id'}, inplace=True, axis=1)

# Write to file
tx.to_parquet(tx_path_out, write_covering_bbox=False,
              geometry_encoding='geoarrow')

# Performance: 1m 29.4s for 39678353 transcripts

In [68]:
# Write counts matrix from transcript assignments (nuclei only)
if bd_type == 'nucleus':

    ad = utils.transcripts_to_anndata(
        tx, cell_label='nucleus_boundaries_id', gene_label='feature_name',
        coordinate_labels=['x_location', 'y_location'])

    ad.write_h5ad(data_dir / 'nucleus_boundaries.h5ad')

# Performance: 7.7s for 274828 x 541 AnnData